# Recommendation & Accuracy 

GOALS:::

Recommendation Strategies
1. k constant: recommend with k many papers
2. k parameter: threshold on the papers

Evaluation Strategies
1. Accuracy metrics
2. Discovery-oriented metrics
"""


# Example

In [1]:
import sys
sys.path.insert(0, '../src')

import preprocess

import random
import matplotlib.pyplot as plt
from collections import defaultdict
%matplotlib inline
from matplotlib import rc
rc('figure', figsize=(16, 8), max_open_warning=False)
from surprise import SVD, SVDpp, NMF
from surprise import Dataset, Reader
from surprise import accuracy
from surprise.model_selection import cross_validate, train_test_split, KFold


###all rights to below for these functions
###http://surprise.readthedocs.io/en/stable/FAQ.html#how-to-get-the-top-n-recommendations-for-each-user
def get_top_n(predictions, n=10):
    '''Return the top-N recommendation for each user from a set of predictions.

    Args:
        predictions(list of Prediction objects): The list of predictions, as
            returned by the test method of an algorithm.
        n(int): The number of recommendation to output for each user. Default
            is 10.

    Returns:
    A dict where keys are user (raw) ids and values are lists of tuples:
        [(raw item id, rating estimation), ...] of size n.
    '''

    # First map the predictions to each user.
    top_n = defaultdict(list)
    for uid, iid, true_r, est, _ in predictions:
        top_n[uid].append((iid, est))

    # Then sort the predictions for each user and retrieve the k highest ones.
    for uid, user_ratings in top_n.items():
        user_ratings.sort(key=lambda x: x[1], reverse=True)
        top_n[uid] = user_ratings[:n]

    return top_n

def precision_recall_at_k(predictions, k=10, threshold=3.5):
    '''Return precision and recall at k metrics for each user.'''

    # First map the predictions to each user.
    user_est_true = defaultdict(list)
    for uid, _, true_r, est, _ in predictions:
        user_est_true[uid].append((est, true_r))

    precisions = dict()
    recalls = dict()
    for uid, user_ratings in user_est_true.items():

        # Sort user ratings by estimated value
        user_ratings.sort(key=lambda x: x[0], reverse=True)

        # Number of relevant items
        n_rel = sum((true_r >= threshold) for (_, true_r) in user_ratings)

        # Number of recommended items in top k
        n_rec_k = sum((est >= threshold) for (est, _) in user_ratings[:k])

        # Number of relevant and recommended items in top k
        n_rel_and_rec_k = sum(((true_r >= threshold) and (est >= threshold))
                              for (est, true_r) in user_ratings[:k])

        # Precision@K: Proportion of recommended items that are relevant
        precisions[uid] = n_rel_and_rec_k / n_rec_k if n_rec_k != 0 else 0

        # Recall@K: Proportion of relevant items that are recommended
        recalls[uid] = n_rel_and_rec_k / n_rel if n_rel != 0 else 0

    return precisions, recalls




In [3]:
# First train an SVD algorithm on the movielens dataset.
data = Dataset.load_builtin('ml-100k')
# trainset = data.build_full_trainset()
trainset, testset = train_test_split(data, test_size=.25)

for epoch in range(10):
    algo = SVDpp(n_epochs=epoch)
    algo.fit(trainset)
    #     testset = trainset.build_anti_testset()
    predictions = algo.test(testset)
    accuracy.rmse(predictions, verbose=True)

# Then predict ratings for all pairs (u, i) that are NOT in the training set.
# testset = trainset.build_anti_testset()
# predictions = algo.test(testset)

# top_n = get_top_n(predictions, n=10)

# # Print the recommended items for each user
# for uid, user_ratings in top_n.items():
#     print(uid, [iid for (iid, _) in user_ratings])

RMSE: 1.1235
RMSE: 0.9898
RMSE: 0.9653
RMSE: 0.9556
RMSE: 0.9502
RMSE: 0.9483
RMSE: 0.9430
RMSE: 0.9390
RMSE: 0.9397
RMSE: 0.9326


In [4]:
data = Dataset.load_builtin('ml-100k')
kf = KFold(n_splits=5)
algo = SVD()

for trainset, testset in kf.split(data):
    algo.fit(trainset)
    predictions = algo.test(testset)
    precisions, recalls = precision_recall_at_k(predictions, k=5, threshold=4)

    # Precision and recall can then be averaged over all users
    print(sum(prec for prec in precisions.values()) / len(precisions))
    print(sum(rec for rec in recalls.values()) / len(recalls))

0.63627034678
0.232776729006
0.639567835636
0.238378885935
0.623983739837
0.236405713292
0.644432624113
0.23655227123
0.613481953291
0.231047307377


# Recommendation System


In [5]:
mydict = preprocess.create_random_subset_paper_paper_data(5000)
data = preprocess.create_surprise_paper_paper_data(mydict)
trainset = data.build_full_trainset()
testset = trainset.build_anti_testset(fill=1)
#### OR
# trainset, testset = train_test_split(data, test_size=.25)
####
algo = SVDpp(n_epochs=100,verbose=True)
algo.fit(trainset)
predictions = algo.test(testset)

k = 10
top_n = get_top_n(predictions, n=k)

# Print the recommended items for each user
for uid, user_ratings in top_n.items():
    print(uid, [iid for (iid, _) in user_ratings], '\n')


 processing epoch 0
 processing epoch 1
 processing epoch 2
 processing epoch 3
 processing epoch 4
 processing epoch 5
 processing epoch 6
 processing epoch 7
 processing epoch 8
 processing epoch 9
 processing epoch 10
 processing epoch 11
 processing epoch 12
 processing epoch 13
 processing epoch 14
 processing epoch 15
 processing epoch 16
 processing epoch 17
 processing epoch 18
 processing epoch 19
 processing epoch 20
 processing epoch 21
 processing epoch 22
 processing epoch 23
 processing epoch 24
 processing epoch 25
 processing epoch 26
 processing epoch 27
 processing epoch 28
 processing epoch 29
 processing epoch 30
 processing epoch 31
 processing epoch 32
 processing epoch 33
 processing epoch 34
 processing epoch 35
 processing epoch 36
 processing epoch 37
 processing epoch 38
 processing epoch 39
 processing epoch 40
 processing epoch 41
 processing epoch 42
 processing epoch 43
 processing epoch 44
 processing epoch 45
 processing epoch 46
 processing epoch 47
 p

d2fd2bf9-d795-413f-ad30-741473136481 ['5f209017-6752-4806-8364-64e2c002ec31', '0cbdbc6b-c13a-4f1a-9511-10b8af5688d9', '9f8150e4-2964-42da-856b-60cf8f3c7ed2', 'c57e5d3a-6951-4f10-8306-86e98d79d54a', 'a0ed15f9-6462-417e-a221-a8cc01fff0b5', '660c5e99-7f66-4f3a-b4f2-bde5e28cc852', 'd209a9e2-78ed-466a-b314-cb405580cef1', 'c455fb04-4566-4648-ad6f-3cf2245e507c', '23b00095-3eac-4115-a9bc-6fdfc1530d71', 'd3812abd-d0b8-4287-84bc-63a68f6b9680'] 

cffc5d8f-30bd-4e82-9a3c-e1b76b5b2219 ['5f209017-6752-4806-8364-64e2c002ec31', '706e2c85-b3e0-4026-81ee-71f1c8c98f92', '8cf429c1-7217-44a3-80b6-a2031bf9ca30', '7f64b879-c8e6-4ce7-920e-cc11f6617b33', 'b5fa0a5b-991b-4552-9bf8-bb78e270b2a7', '43672606-0369-42da-9ae8-d06034c79b6b', '57d8a81b-3058-4c65-88a9-c3b611ffa20a', '11a5a19d-8776-419d-83a7-3f913f9e1c39', '9ff6f1ec-dde8-45d2-839f-6593f0b04094', '01ee56e7-420d-42e4-9fc7-6aa5da07640e'] 

c57e5d3a-6951-4f10-8306-86e98d79d54a ['939e176e-4d1b-4982-adc5-b93f7616b7e6', '5552eb7d-4710-4813-bc4e-f4e6eec1ba90', '5

In [12]:
precisions, recalls = precision_recall_at_k(predictions, k=10, threshold=0.5)

# Precision and recall can then be averaged over all users
print(sum(prec for prec in precisions.values()) / len(precisions))
print(sum(rec for rec in recalls.values()) / len(recalls))

0.0
0.0


In [7]:
for epoch in range(1):
    algo.fit(trainset)
    testset = trainset.build_anti_testset()
    predictions = algo.test(testset)
    

 processing epoch 0
 processing epoch 1
 processing epoch 2
 processing epoch 3
 processing epoch 4
 processing epoch 5
 processing epoch 6
 processing epoch 7
 processing epoch 8
 processing epoch 9
 processing epoch 10
 processing epoch 11
 processing epoch 12
 processing epoch 13
 processing epoch 14
 processing epoch 15
 processing epoch 16
 processing epoch 17
 processing epoch 18
 processing epoch 19
 processing epoch 20
 processing epoch 21
 processing epoch 22
 processing epoch 23
 processing epoch 24
 processing epoch 25
 processing epoch 26
 processing epoch 27
 processing epoch 28
 processing epoch 29
 processing epoch 30
 processing epoch 31
 processing epoch 32
 processing epoch 33
 processing epoch 34
 processing epoch 35
 processing epoch 36
 processing epoch 37
 processing epoch 38
 processing epoch 39
 processing epoch 40
 processing epoch 41
 processing epoch 42
 processing epoch 43
 processing epoch 44
 processing epoch 45
 processing epoch 46
 processing epoch 47
 p

In [1]:
import sys
sys.path.insert(0, '../src')
import preprocess # This line only works with our settings in this colab notebook.
import random
import matplotlib.pyplot as plt
from collections import defaultdict
%matplotlib inline
from matplotlib import rc
rc('figure', figsize=(16, 8), max_open_warning=False)
from surprise import SVD, SVDpp, NMF
from surprise import Dataset
from surprise.model_selection import cross_validate
from surprise import accuracy
from surprise.model_selection import train_test_split
import pandas as pd
import numpy as np


### Below tic toc from https://stackoverflow.com/questions/5849800/tic-toc-functions-analog-in-python
import time
def TicTocGenerator():
    # Generator that returns time differences
    ti = 0           # initial time
    tf = time.time() # final time
    while True:
        ti = tf
        tf = time.time()
        yield tf-ti # returns the time difference

TicToc = TicTocGenerator() # create an instance of the TicTocGen generator

# This will be the main function through which we define both tic() and toc()
def toc(tempBool=True):
    # Prints the time difference yielded by generator instance TicToc
    tempTimeInterval = next(TicToc)
    if tempBool:
        print( "Elapsed time: %f seconds.\n" %tempTimeInterval )

def tic():
    # Records a time in TicToc, marks the beginning of a time interval
    toc(False)

In [2]:
# tic()
# mydict = preprocess.create_user_paper_dict(datadir="../dblp-ref",debug=True)
mydict = preprocess.create_random_subset_user_paper_data(5000,debug=True)
# toc()
 
mydict


{'Evangelos A. Coutsias': defaultdict(None,
             {'658fc276-8f8e-4ab3-a481-bd55210fb5e8': 1,
              '81d6a598-396a-47df-bcc8-66f6bd6950d7': 1,
              '921283ab-a43f-4f8b-942f-0980cbf29892': 1,
              '9a7ad010-b533-4278-b736-05431eb7360b': 1,
              'f58d9855-4af6-4545-a695-e45ea00b9186': 1}),
 'Franck Patrick Vidal': defaultdict(None,
             {'00eb9413-42a1-4405-8ad7-ada2719577df': 1,
              '03a2186f-8f49-46b0-80db-9a3560a14edd': 1,
              '09bb8a4c-e8fc-4c33-835c-1524020298b1': 1,
              '172f9f68-8417-43bb-8fe5-b377d569f6b6': 1,
              '1a12c567-cc4c-4d16-aa93-0b19c7f9ded6': 1,
              '202ba752-67ea-4121-8b8a-78b87c45f42e': 1,
              '26f4a713-7dee-47c6-9086-4dd9f12907f4': 1,
              '2d3e0f45-fb15-4084-9948-66abfcf384da': 1,
              '467838c8-968a-4c56-9f82-f897a2717072': 1,
              '4eadbb41-56a9-4bc7-bcf6-2bf0a8b04544': 1,
              '6de33977-0c95-4ae5-8cd5-160193f4e80f': 1,

In [4]:
testsize = 0.25
all_nonzero_entries = set()
users_to_be_prevented = set()
papers_to_be_prevented = set()

for user in mydict:
    if len(mydict[user]) < 2:
        users_to_be_prevented.add(user)
    for paper in mydict[user]:
        all_nonzero_entries.add((user,paper,mydict[user][paper]))
        cited_by_how_many = 0
        for userrr in mydict:
            if paper in mydict[userrr]:
                cited_by_how_many += 1
        if cited_by_how_many < 2:
            papers_to_be_prevented.add(paper)

entries_to_be_prevented = set()
for entry in all_nonzero_entries:
    if entry[0] in users_to_be_prevented or entry[1] in papers_to_be_prevented:
        entries_to_be_prevented.add(entry)

splittables = all_nonzero_entries - entries_to_be_prevented
L = len(all_nonzero_entries)
M = len(entries_to_be_prevented)
x = M/L

In [5]:
x

0.6315350968548219

In [25]:
#People who don't cite any paper.

freesouls = []
for author in mydict:
  if len(mydict[author]) == 0:
    freesouls.append(author)
    
len(freesouls)
len(mydict)

3079007

In [42]:
baby = {k:k for k in np.arange(0,11)}

In [13]:
del baby[0]

In [25]:
baby

{11: 11, 12: 12, 13: 13, 14: 14, 15: 15, 16: 16, 17: 17, 18: 18, 19: 19}

In [5]:
baby = set()

In [7]:
baby.add(0)

In [34]:
baby = (3,4,"yo")

In [38]:
baby[2]

'yo'

In [22]:
baby1 = set(baby)
# baby1.update(baby.keys())

In [32]:
baby1.update(baby)

In [48]:
baby = set(np.arange(0,21))
baby

{0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20}

In [51]:
random.sample(baby,5)

[3, 9, 4, 5, 11]